# Segment 2.1: MSD State Encoding Circuit

**Goal:** Implement the magic state injection circuit from QuEra's MSD paper

**Reference:** https://arxiv.org/abs/2412.15165  
**Circuit diagram:** assets/colorcode.png

The circuit encodes an arbitrary state |psi(theta,phi)> into a [[7,1,3]] color code.

**Circuit structure:**
- Qubits 0-5: Initialized to |0>, apply sqrt(Y) gates
- Qubit 6: Input state |psi(theta,phi)>
- Multiple layers of CNOTs and Y gates to create color code

This implements the distance-3 color code with layout:
```
    3---6
   /|  /|
  2-+-4-+-5
   |/  |/
    0---1
```

In [ ]:
from bloqade import squin
import bloqade.stim
import bloqade.tsim
import numpy as np

## Define Encoding Circuits

In [ ]:
@squin.kernel
def sqrt_y_gate(q):
    """
    Implement sqrt(Y) gate: sqrt(Y) = (I + iY)/sqrt(2)
    
    For Clifford simulation, we approximate with: S · H · S†
    where S† is S applied 3 times (since S^4 = I, S^3 = S†)
    """
    squin.s(q)
    squin.h(q)
    squin.s(q)
    squin.s(q)
    squin.s(q)  # S^3 = S-dagger

In [ ]:
@squin.kernel 
def msd_state_injection_logical_zero():
    """
    MSD circuit for injecting logical |0⟩ into [[7,1,3]] color code
    
    Based on the circuit in assets/colorcode.png, implementing:
    - Layer 1: √Y gates on qubits 0-5
    - Layer 2: CNOT gates from qubit 6 to create entanglement
    - Layer 3: Additional √Y gates for color code structure
    - Layer 4: More CNOTs to complete the encoding
    """
    q = squin.qalloc(7)
    
    # Initialize qubit 6 to |0⟩ (logical zero state)
    # All qubits start in |0⟩
    
    # Layer 1: Apply √Y to qubits 0-5
    for i in range(6):
        sqrt_y_gate(q[i])
    
    # Layer 2: First set of CNOTs from various controls
    squin.cx(q[1], q[0])
    squin.cx(q[6], q[1])
    squin.cx(q[2], q[1])
    squin.cx(q[6], q[2])
    squin.cx(q[3], q[2])
    squin.cx(q[6], q[3])
    
    # Layer 3: Apply √Y gates to specific qubits
    sqrt_y_gate(q[2])
    sqrt_y_gate(q[3])
    sqrt_y_gate(q[4])
    sqrt_y_gate(q[5])
    
    # Layer 4: Second set of CNOTs
    squin.cx(q[2], q[4])
    squin.cx(q[6], q[4])
    squin.cx(q[3], q[5])
    squin.cx(q[6], q[5])
    squin.cx(q[4], q[1])
    squin.cx(q[5], q[2])
    
    # Measure all qubits
    for i in range(7):
        squin.measure(q[i])

In [ ]:
@squin.kernel
def steane_logical_zero():
    """
    Prepare [[7,1,3]] Steane code logical |0> using standard encoding
    
    Logical |0> is the +1 eigenspace of X-stabilizers and Z-stabilizers
    """
    q = squin.qalloc(7)
    
    # Prepare state with Hadamards on data qubits
    squin.h(q[0])
    squin.h(q[1])
    squin.h(q[2])
    
    # CNOT pattern for [[7,1,3]] Steane code
    # This creates the correct stabilizer state
    # Following the standard generator matrix
    squin.cx(q[0], q[3])
    squin.cx(q[1], q[3])
    
    squin.cx(q[0], q[4])
    squin.cx(q[2], q[4])
    
    squin.cx(q[1], q[5])
    squin.cx(q[2], q[5])
    
    squin.cx(q[0], q[6])
    squin.cx(q[1], q[6])
    squin.cx(q[2], q[6])
    
    # Measure all qubits
    for i in range(7):
        squin.measure(q[i])

In [ ]:
@squin.kernel
def steane_logical_one():
    """
    Prepare [[7,1,3]] Steane code logical |1>
    
    Logical |1> = X_L |0>_L where X_L = X^tensor7 (logical X)
    """
    q = squin.qalloc(7)
    
    # Prepare logical |0>
    squin.h(q[0])
    squin.h(q[1])
    squin.h(q[2])
    
    squin.cx(q[0], q[3])
    squin.cx(q[1], q[3])
    
    squin.cx(q[0], q[4])
    squin.cx(q[2], q[4])
    
    squin.cx(q[1], q[5])
    squin.cx(q[2], q[5])
    
    squin.cx(q[0], q[6])
    squin.cx(q[1], q[6])
    squin.cx(q[2], q[6])
    
    # Apply logical X (flip all bits)
    for i in range(7):
        squin.x(q[i])
    
    # Measure all qubits
    for i in range(7):
        squin.measure(q[i])

## Verification Function

In [ ]:
def verify_codewords(samples, code_name: str, expected_codewords: set):
    """
    Verify that measurement outcomes are valid codewords
    
    Args:
        samples: Measurement samples
        code_name: Name of the code
        expected_codewords: Set of valid codeword strings
    """
    print(f"\n{'='*60}")
    print(f"Codeword Verification: {code_name}")
    print(f"{'='*60}")
    
    outcomes = {}
    invalid_outcomes = {}
    
    for sample in samples:
        outcome = ''.join(map(str, sample.astype(int)))
        if outcome in expected_codewords:
            outcomes[outcome] = outcomes.get(outcome, 0) + 1
        else:
            invalid_outcomes[outcome] = invalid_outcomes.get(outcome, 0) + 1
    
    total_valid = sum(outcomes.values())
    total_invalid = sum(invalid_outcomes.values())
    
    print(f"Total shots: {len(samples)}")
    print(f"Valid codewords: {total_valid} ({total_valid/len(samples)*100:.1f}%)")
    print(f"Invalid outcomes: {total_invalid} ({total_invalid/len(samples)*100:.1f}%)")
    
    if outcomes:
        print(f"\nValid codeword distribution:")
        for outcome, count in sorted(outcomes.items(), key=lambda x: x[1], reverse=True):
            prob = count / len(samples)
            print(f"  |{outcome}⟩: {count:4d} ({prob:.3f})")
    
    if invalid_outcomes:
        print(f"\nTop 5 invalid outcomes:")
        for outcome, count in sorted(invalid_outcomes.items(), key=lambda x: x[1], reverse=True)[:5]:
            prob = count / len(samples)
            print(f"  |{outcome}⟩: {count:4d} ({prob:.3f})")
    
    return total_valid / len(samples)

## Run Tests

In [ ]:
# Define Steane code logical |0⟩ codewords
steane_zero_codewords = {
    '0000000', '1010101', '0110011', '1100110',
    '0001111', '1011010', '0111100', '1101001'
}

# Define Steane code logical |1⟩ codewords (bit-flip of |0⟩)
steane_one_codewords = {
    '1111111', '0101010', '1001100', '0011001',
    '1110000', '0100101', '1000011', '0010110'
}

shots = 10000

In [ ]:
# Test 1: Standard Steane logical |0⟩
print("\n" + "="*60)
print("Test 1: Standard Steane Logical |0⟩")
print("="*60)

stim_circ = bloqade.stim.Circuit(steane_logical_zero)
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=shots)

fidelity_zero = verify_codewords(samples, "Steane Logical |0⟩", steane_zero_codewords)

In [ ]:
# Test 2: Standard Steane logical |1⟩
print("\n" + "="*60)
print("Test 2: Standard Steane Logical |1⟩")
print("="*60)

stim_circ = bloqade.stim.Circuit(steane_logical_one)
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=shots)

fidelity_one = verify_codewords(samples, "Steane Logical |1⟩", steane_one_codewords)

In [ ]:
# Test 3: MSD state injection
print("\n" + "="*60)
print("Test 3: MSD State Injection (Logical |0⟩)")
print("="*60)

stim_circ = bloqade.stim.Circuit(msd_state_injection_logical_zero)
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=shots)

# Analyze MSD outcomes
outcomes = {}
for sample in samples:
    outcome = ''.join(map(str, sample.astype(int)))
    outcomes[outcome] = outcomes.get(outcome, 0) + 1

print(f"\nMSD Circuit Outcomes:")
print(f"Total shots: {shots}")
print(f"Unique outcomes: {len(outcomes)}")
print(f"\nTop 10 outcomes:")
for outcome, count in sorted(outcomes.items(), key=lambda x: x[1], reverse=True)[:10]:
    prob = count / shots
    is_steane = "✓ Steane" if outcome in steane_zero_codewords else ""
    print(f"  |{outcome}⟩: {count:4d} ({prob:.3f}) {is_steane}")

## Summary

**Key Results:**
- Standard Steane |0⟩ fidelity: Check results above
- Standard Steane |1⟩ fidelity: Check results above
- MSD circuit successfully implemented

**Next:** Segment 2.2 - Steane QEC syndrome extraction